In [40]:
from lxml import etree as ET
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib



In [55]:
%load_ext autoreload
%autoreload 2
''
config = RClib.loadconfigs('.\config.json')
scandf = RClib.provide_scandf(inputdirectory=config['workspace'], imageformat = '*.dng')
previousscandf_path = Path(config['workspace']) / 'rcprocessingdf.pkl'
if previousscandf_path.is_file() and config['resume_processing']:
    previousscandf = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
    previousscandf = previousscandf.set_index('id',drop=False)
    scandf = scandf.set_index('id',drop=False)
    print(previousscandf.columns)
    scandf  = pd.concat([scandf , previousscandf[previousscandf.columns.difference(scandf.columns)]], join='outer', axis=1)
    scandf.update(previousscandf)

print(scandf.columns)

scandf= scandf.apply(RClib.defineRawTherapeeOutput, foldername='02_PNG', axis=1)
scandf= scandf.apply(RClib.defineRealityCaptureOutput, foldername='03_RC', axis=1)
#rcsettings_template = pd.read_csv('.\\allRCsettings_template.csv', sep=';', header=0) 
#print(rcsettings_template)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Index(['RCoutputfolder', 'RToutputfolder', 'align-rccmdpath',
       'commandlist_align', 'commandlist_alignsettings', 'id', 'imagedf',
       'list_dev-img_path', 'pp3file', 'rcproj_path', 'scan_dir'],
      dtype='object')
Index(['id', 'scan_dir', 'pp3file', 'imagedf', 'RCoutputfolder',
       'RToutputfolder', 'align-rccmdpath', 'commandlist_align',
       'commandlist_alignsettings', 'list_dev-img_path', 'rcproj_path'],
      dtype='object')


In [57]:

scandf2=pd.DataFrame()
for index,scan in scandf.iterrows():
    scan['processingstatus'] = []
    scan['imagedf'] = scan['imagedf'].apply(RClib.developwithRawTherapee, pp3filepath=scan['pp3file'][0], outputfolderpath=scan['RToutputfolder'], axis=1) 
    scan = RClib.makeImagelist(scan, imagelistname=scan['id'])
    scan = RClib.createRCproject(scan)

    scan['commandlist_alignsettings']=Path(config['alignmentsettings'])
    RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_alignsettings' )
    scan['commandlist_align'] = []
    scan['commandlist_align'].append('-add ' + str(scan['list_dev-img_path']))
    scan['commandlist_align'].append('-detectMarkers')
    scan['commandlist_align'].append('-align')
    scan['commandlist_align'].append('-exportReport ' + str(Path(scan['RCoutputfolder'] / Path(scan['id']+'.html '))) + config['alignmentreport_template'])
    scan['commandlist_align'].append('-save')
    scan['commandlist_align'].append('-quit')
    scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_align', rccmdpathfield='align-rccmdpath')
    RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='align-rccmdpath' )
    scan['processingstatus'].append({'alignment_report': Path(scan['RCoutputfolder'] / Path(scan['id']+'.html '))})
    scandf2 = scandf2.append(scan)
#pd.write_csv(Path(config['workspace']) / 'rcprocessingdf.csv', sep=';')


E:\RCprocessing_test\PG_boat91_test\03_RC\commandlist_align.rccmd
E:\RCprocessing_test\PG_boat_test\03_RC\commandlist_align.rccmd


In [58]:
print(scandf2)
scandf2.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

                                           RCoutputfolder  \
PG_boat91_test  E:\RCprocessing_test\PG_boat91_test\03_RC   
PG_boat_test      E:\RCprocessing_test\PG_boat_test\03_RC   

                                            RToutputfolder  \
PG_boat91_test  E:\RCprocessing_test\PG_boat91_test\02_PNG   
PG_boat_test      E:\RCprocessing_test\PG_boat_test\02_PNG   

                                                  align-rccmdpath  \
PG_boat91_test  E:\RCprocessing_test\PG_boat91_test\03_RC\comm...   
PG_boat_test    E:\RCprocessing_test\PG_boat_test\03_RC\comman...   

                                                commandlist_align  \
PG_boat91_test  [-add E:\RCprocessing_test\PG_boat91_test\03_R...   
PG_boat_test    [-add E:\RCprocessing_test\PG_boat_test\03_RC\...   

                                        commandlist_alignsettings  \
PG_boat91_test  E:\GitHub\RCprocessing\RCsettings_alignment.rccmd   
PG_boat_test    E:\GitHub\RCprocessing\RCsettings_alignment.rccmd   

     

In [59]:
scandf3 = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
print(scandf3)

                                           RCoutputfolder  \
PG_boat91_test  E:\RCprocessing_test\PG_boat91_test\03_RC   
PG_boat_test      E:\RCprocessing_test\PG_boat_test\03_RC   

                                            RToutputfolder  \
PG_boat91_test  E:\RCprocessing_test\PG_boat91_test\02_PNG   
PG_boat_test      E:\RCprocessing_test\PG_boat_test\02_PNG   

                                                  align-rccmdpath  \
PG_boat91_test  E:\RCprocessing_test\PG_boat91_test\03_RC\comm...   
PG_boat_test    E:\RCprocessing_test\PG_boat_test\03_RC\comman...   

                                                commandlist_align  \
PG_boat91_test  [-add E:\RCprocessing_test\PG_boat91_test\03_R...   
PG_boat_test    [-add E:\RCprocessing_test\PG_boat_test\03_RC\...   

                                        commandlist_alignsettings  \
PG_boat91_test  E:\GitHub\RCprocessing\RCsettings_alignment.rccmd   
PG_boat_test    E:\GitHub\RCprocessing\RCsettings_alignment.rccmd   

     